# Fetching and Loading Environmental Data

In this tutorial, we will use Kadlu to retrieve environmental data from online sources and load the data into numpy arrays for further processing.

To start with, we import all necessary python modules, functions, etc.

In [1]:
from datetime import datetime
from kadlu import chs, era5, hycom, wwiii, source_map, data_util

### Quick start guide

With Kadlu, environmental data can be retrieved and loaded with a single line of code. Here, we demonstrate how to obtain modelled, surface salinity data from HYCOM for the geographic region $47^{\circ}$N to $49^{\circ}$N and $-63^{\circ}$W to $-61^{\circ}$W for the first week of January 2013.     

In [2]:
# fetch and load salinity (g/kg salt in water)
salinity, lat, lon, epoch, depth = hycom().load_salinity(
        south=47, west=-63, 
        north=49, east=-61, 
        bottom=0, top=0,
        start=datetime(2013, 1, 1), end=datetime(2013, 1, 7))

Note how the arguments `bottom` and `top` are both set to `0`, thereby selecting only data at a depth of 0 m, i.e., at the surface. Note also that we use the [datetime](https://docs.python.org/2/library/datetime.html) package to specify dates and times.

The `load_salinity` method produced flattened numpy arrays, the length of which corresponds to the number of data points in the selected geographic region, depth range, and temporal window. 

In [3]:
# print the first 10 values of each array

print(lat[0:10])      # latitude (degrees north)
print(lon[0:10])      # longitude (degrees west)
print(depth[0:10])    # depth (meters)
print(epoch[0:10])     # time (hours since 00:00:00 on 1 January 2000)
print(salinity[0:10]) # ocean salt content (g/kg)

[47. 47. 47. 47. 47. 47. 47. 47. 47. 47.]
[-62.96002197 -62.88000488 -62.79998779 -62.7199707  -62.64001465
 -62.55999756 -62.47998047 -62.40002441 -62.32000732 -62.23999023]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[113991. 113991. 113991. 113991. 113991. 113991. 113991. 113991. 113991.
 113991.]
[30.791 30.886 30.92  30.903 30.874 30.853 30.817 30.747 30.714 30.636]


We can use the `epoch_2_dt` function from Kadlu's `data_util` module to convert the time values into a more human-friendly date-time format, 

In [9]:
print(data_util.epoch_2_dt(epoch[0]))

2013-01-01 15:00:00


### Data sources
Kadlu includes functionality to load data from a variety of different data sources. For a high level overview, print the source_map:

In [6]:
print(source_map)


    CHS   (Canadian Hydrography Service)
          load_bathymetry:          bathymetric data in Canada's waterways. variable resolution 

    ERA5  (Global environmental dataset from Copernicus Climate Data Store)
          load_windwaveswellheight: combined height of wind, waves, and swell. metres
          load_wavedirection:       mean wave direction, degrees
          load_waveperiod:          mean wave period, seconds
          load_wind_uv:             wind speed computed as sqrt(u^2 + v^2) / 2, where u, v are direction vectors
          load_wind_u:              wind speed coordinate U-vector, m/s
          load_wind_v:              wind speed coordinate V-vector, m/s 

    HYCOM (Hybrid Coordinate Ocean Model)
          load_salinity:            g/kg salt in water
          load_temp:                degrees celsius
          load_water_uv:            ocean current computed as sqrt(u^2 + v^2) / 2, where u, v are direction vectors
          load_water_u:             ocean curre

for more information on a specific source, print the class object:

In [7]:
print(hycom())

Native hycom .[ab] data converted to NetCDF at the Naval
Research Laboratory, interpolated to 0.08° grid between
40°S-40°N (0.04° poleward) containing 40 z-levels.
Availability: 1994 to 2015
	https://www.hycom.org/data/glbv0pt08

function input arguments:
	(south, north, west, east, start, end, top, bottom)

class functions:
	load_salinity
	load_temp
	load_water_u
	load_water_uv
	load_water_v



keyword arguments can be passed as a dictionary when using the same load arguments for multiple datatypes

In [8]:
kwargs = dict(
        south=47, west=-63, 
        north=49, east=-61, 
        bottom=0, top=0,
        start=datetime(2013, 1, 1), end=datetime(2013, 1, 7))

bathy, lat1, lon1 = chs().load_bathymetry(**kwargs)
waveheight, lat2, lon2, epoch2 = era5().load_windwaveswellheight(**kwargs)

CHS bathymetry: processing 16 files
CHS CA2_4600N06100W.tif bathymetry: processed and inserted 73596 rows. 928405 null values removed, 0 duplicate rows ignored
CHS CA2_4600N06200W.tif bathymetry: processed and inserted 0 rows. 938631 null values removed, 63370 duplicate rows ignored
CHS CA2_4600N06300W.tif bathymetry: processed and inserted 0 rows. 901841 null values removed, 100160 duplicate rows ignored
CHS CA2_4600N06400W.tif bathymetry: processed and inserted 0 rows. 844720 null values removed, 157281 duplicate rows ignored
CHS CA2_4700N06100W.tif bathymetry: processed and inserted 23357 rows. 978644 null values removed, 0 duplicate rows ignored
CHS CA2_4700N06200W.tif bathymetry: processed and inserted 0 rows. 528221 null values removed, 473780 duplicate rows ignored
CHS CA2_4700N06300W.tif bathymetry: processed and inserted 0 rows. 910618 null values removed, 91383 duplicate rows ignored
CHS CA2_4700N06400W.tif bathymetry: processed and inserted 0 rows. 994983 null values removed

2020-03-03 13:20:51,037 INFO Welcome to the CDS
2020-03-03 13:20:51,040 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2020-03-03 13:20:54,053 INFO Request is queued
2020-03-03 13:20:56,857 INFO Request is running
2020-03-03 13:21:02,794 INFO Request is completed
2020-03-03 13:21:02,796 INFO Downloading http://136.156.132.210/cache-compute-0005/cache/data2/adaptor.mars.internal-1583256056.1745522-19056-8-6f74b241-1567-4cf4-8bea-37fbb68846ca.grib to /storage/ERA5_reanalysis_significant_height_of_combined_wind_waves_and_swell_2013-01-01.grb2 (7.3M)
2020-03-03 13:21:05,994 INFO Download rate 2.3M/s
2020-03-03 13:21:11,965 INFO Welcome to the CDS
2020-03-03 13:21:11,966 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels
2020-03-03 13:21:13,313 INFO Request is queued
2020-03-03 13:21:24,485 INFO Request is running
2020-03-03 13:21:29,701 INFO Request is completed
2020-03-03 13:21: